#### **Regression-Analysis-Team-Project** by Team 1

##### Importing necessary libraries

In [ ]:
####### what's in startup file #######
import numpy as np
import scipy as sp
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import sklearn as sk

import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
sns.set()
sns.set_style("whitegrid")
sns.set_color_codes()

import xgboost as xgb
from numpy import loadtxt
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore')

In [1]:
####### what's not in startup file #######
from scipy import stats

# visualization in jupyter
%matplotlib inline   

# ignore warning messeges
import warnings
warnings.filterwarnings('ignore')

import time

##### Importing data
- Properties Data

In [2]:
merged = pd.read_csv('data/properties_2016.csv')

In [3]:
merged_des = merged.describe(include='all').transpose()
merged_des["missing_ratio"] = 1-(merged_des["count"]/merged.shape[0])
merged_des["dtype"] = merged.dtypes
merged_des.reset_index().tail()

,index,count,unique,top,freq,mean,std,min,25%,50%,75%,max,missing_ratio,dtype
53,landtaxvaluedollarcnt,2.91748e+06,NaN,NaN,NaN,252478,445013,1,74836,167042,306918,9.02462e+07,0.0226895,float64
54,taxamount,2.95397e+06,NaN,NaN,NaN,5377.61,9183.11,1.34,2461.07,3991.78,6201.01,3.45886e+06,0.0104683,float64
55,taxdelinquencyflag,56462,1,Y,56462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.981086,object
56,taxdelinquencyyear,56464,NaN,NaN,NaN,13.8924,2.58101,0,14,14,15,99,0.981085,float64
57,censustractandblock,2.91009e+06,NaN,NaN,NaN,6.04843e+13,3.24903e+11,-1,6.0374e+13,6.03757e+13,6.05904e+13,4.8303e+14,0.025166,float64


##### Data의 format 등에 먼저 전처리를 해줄 수 있는 부분

In [4]:
# taxdelinquency는 "Y"를 True로 바꾸어 줌 (다른 dummy 변수들은 1이나 True로 되어 있음)
merged['taxdelinquencyflag'] = merged['taxdelinquencyflag'].replace('Y',True)
merged['taxdelinquencyflag'] = merged['taxdelinquencyflag'].fillna(False)

# taxdelinquencyyear는 4자리의 년도로 format을 맞춰줌
merged["taxdelinquencyyear"] = merged["taxdelinquencyyear"].apply(lambda x: x + 2000 if x <= 17 else x + 1900)

# 위도 경도를 원래 단위로 맞춰줌
merged["latitude"] = merged["latitude"]/1000000
merged["longitude"] = merged["longitude"]/1000000

In [9]:
merged["transaction_year"] = 2016

##### categorical / numerical variable 구분하기

In [10]:
vars = merged_des.reset_index()["index"].tolist()

cat_vars = ['transaction_year'
            'rawcensustractandblock', 'censustractandblock', 'fips',
            'regionidcity', 'regionidcounty', 'regionidneighborhood', 'regionidzip', 
            'architecturalstyletypeid', 'decktypeid', 'buildingclasstypeid', 'storytypeid',
            'typeconstructiontypeid', 'propertycountylandusecode', 'propertylandusetypeid',
            'propertyzoningdesc', 'taxdelinquencyflag', 'taxdelinquencyyear', 'assessmentyear',
            'heatingorsystemtypeid', 'airconditioningtypeid', 'hashottuborspa', 'fireplaceflag',
            'pooltypeid10', 'pooltypeid2', 'pooltypeid7']

num_vars = [i for i in vars if i not in cat_vars]

num_vars.remove('parcelid')

print("number of categorical variables:", len(cat_vars))
print("number of numerical variables:", len(num_vars))

number of categorical variables: 25
number of numerical variables: 33


##### 카테고리 변수 중 float type이 많음 → 필요할 경우 str으로 바꿔주기
- str: 'architecturalstyletypeid', 'rawcensustractandblock', 'censustractandblock', 'regionidcounty', 'fips', 'regionidcity', 'regionidcounty', 'regionidneighborhood', 'decktypeid', 'buildingclasstypeid', 'storytypeid', 'typeconstructiontypeid', 'propertylandusetypeid', 'heatingorsystemtypeid', 'airconditioningtypeid'
    - 일단 년도 변수는 계산 등의 조작 가능성을 고려해 바꿔주지 않음: 'assessmentyear', 'taxdelinquencyyear'
- boolean: 'fireplaceflag', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7', 'hashottuborspa'

In [11]:
def cat_to_str(column_name):
    merged[column_name] = merged[column_name].astype("str")
    merged[column_name][merged[column_name] == 'nan'] = np.nan

In [12]:
ls_c_to_s = ['architecturalstyletypeid', 'rawcensustractandblock', 'censustractandblock',
             'regionidcounty', 'fips', 'regionidcity', 'regionidcounty', 'regionidneighborhood',
             'decktypeid', 'buildingclasstypeid', 'storytypeid', 'typeconstructiontypeid',
             'propertylandusetypeid', 'heatingorsystemtypeid', 'airconditioningtypeid']

for var in ls_c_to_s:
    cat_to_str(var)

In [13]:
# dummy 변수를 True/False로 인코딩
def cat_to_bln(column_name):
    merged[column_name] = merged[column_name].replace(1,True)
    merged[column_name] = merged[column_name].fillna(False)

In [14]:
c_to_b = ['fireplaceflag', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7', 'hashottuborspa']

for var in c_to_b:
    cat_to_bln(var)

In [15]:
merged_des = merged.describe(include='all').transpose()
merged_des["missing_ratio"] = 1-(merged_des["count"]/merged.shape[0])
merged_des["dtype"] = merged.dtypes
merged_des.tail()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max,missing_ratio,dtype
parcelid,2.98522e+06,NaN,NaN,NaN,1.33259e+07,7.90997e+06,1.07117e+07,1.16437e+07,1.25451e+07,1.40971e+07,1.69602e+08,0,int64
airconditioningtypeid,811519,7,1.0,742364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.728154,object
architecturalstyletypeid,6061,8,7.0,5251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.99797,object
basementsqft,1628,NaN,NaN,NaN,646.883,538.793,20,272,534,847.25,8516,0.999455,float64
bathroomcnt,2.97376e+06,NaN,NaN,NaN,2.20914,1.07775,0,2,2,3,20,0.00383959,float64
bedroomcnt,2.97377e+06,NaN,NaN,NaN,3.08895,1.27586,0,2,3,4,20,0.00383557,float64
buildingclasstypeid,12629,5,4.0,9265,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.995769,object
buildingqualitytypeid,1.93849e+06,NaN,NaN,NaN,5.78479,1.80535,1,4,7,7,12,0.350637,float64
calculatedbathnbr,2.85630e+06,NaN,NaN,NaN,2.29926,1.00074,1,2,2,3,20,0.0431835,float64
decktypeid,17096,1,66.0,17096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.994273,object


## 2.2 Exploring Target Variable: logerror

In [ ]:
# censustractandblock & rawcensustractandblock의 5-10자리 부분 중 첫째 자리로 카테고리화 (카테고리 수를 줄임)
merged["census_3"] = merged["censustractandblock"].astype("str").apply(lambda x: x[4:5])
merged["census_3"][merged["census_3"] == ''] = np.nan
census_3 = pd.DataFrame(merged["census_3"])
census_3 = census_3.groupby("census_3").size().reset_index(name='counts')

merged["raw_census_3"] = merged["rawcensustractandblock"].astype("str").apply(lambda x: x[4:5])
merged["raw_census_3"][merged["raw_census_3"] == ''] = np.nan
raw_census_3 = pd.DataFrame(merged["raw_census_3"])
raw_census_3 = raw_census_3.groupby("raw_census_3").size().reset_index(name='counts')

In [ ]:
# censustractandblock & rawcensustractandblock의 1-5자리로 카테고리화
merged["census_4"] = merged["censustractandblock"].astype("str").apply(lambda x: x[:5])
merged["census_4"][merged["census_4"] == 'nan'] = np.nan
census_4 = pd.DataFrame(merged["census_4"])
census_4 = census_4.groupby("census_4").size().reset_index(name='counts')

merged["raw_census_4"] = merged["rawcensustractandblock"].astype("str").apply(lambda x: x[:5])
merged["raw_census_4"][merged["raw_census_4"] == 'nan'] = np.nan
raw_census_4 = pd.DataFrame(merged["raw_census_4"])
raw_census_4 = raw_census_4.groupby("raw_census_4").size().reset_index(name='counts')

In [ ]:
### 입력오류라고 보고 NaN값 처리
merged["regionidzip"][merged["regionidzip"]>=100000] = np.nan

In [ ]:
# 첫글자로 새로운 변수 만들기 (첫 글자로 카테고리화)
merged["propertyzoningdesc_1"] = merged["propertyzoningdesc"].astype("str").apply(lambda x: x[:1])
merged["propertyzoningdesc_1"][merged["propertyzoningdesc_1"] == "n"] = np.nan

In [ ]:
# 세번째 글자로 새로운 변수 만들기
merged["propertyzoningdesc_3"] = merged["propertyzoningdesc"].astype("str").apply(
    lambda x: x.replace("&", "").replace("-", "").replace(" ", ""))
merged["propertyzoningdesc_3"] = merged["propertyzoningdesc_3"].apply(lambda x: x[2:3])
merged["propertyzoningdesc_3"][merged["propertyzoningdesc_3"] == "n"] = np.nan
merged["propertyzoningdesc_3"][merged["propertyzoningdesc_3"] == ""] = np.nan

In [ ]:
merged["taxdelinquencyyear_past"] = 2016 - merged["taxdelinquencyyear"]

## **3. Feature Engineering**

### 3.1 Missing Values

#### 3.1.1 Status of missing values
- 데이터에 missing value가 상당히 많음, missing value의 비중을 높은 순으로 살펴보자

In [ ]:
merged_missing = pd.DataFrame(merged_des["missing_ratio"].sort_values(ascending=False))
merged_missing.reset_index()

#### 3.1.3 Missing Values processing

#### 일부 변수 추가 후 categorical / numerical 새로 분류
- poolcnt는 dummy variable로 재분류함 (True/False)

In [ ]:
merged_1 = merged.copy()
merged_1.tail()

In [ ]:
vars = merged_des.reset_index()["index"].tolist()
vars.append("taxdelinquencyyear_past")

cat_vars = ['fips', 'rawcensustractandblock', 'censustractandblock', "raw_census_1", "census_1",
            "raw_census_2", "census_2", "raw_census_3", "census_3", "raw_census_4", "census_4",
            'regionidcounty', 'regionidcity', 'regionidzip', 'regionidneighborhood', 'architecturalstyletypeid',
            'decktypeid', 'buildingqualitytypeid', 'buildingclasstypeid', 
            'storytypeid', 'typeconstructiontypeid', 'propertycountylandusecode', 'propertylandusetypeid',
            'propertyzoningdesc', "propertyzoningdesc_1", "propertyzoningdesc_3",
            'taxdelinquencyflag', 'taxdelinquencyyear', 'assessmentyear',
            'heatingorsystemtypeid', 'airconditioningtypeid', 'hashottuborspa', 'fireplaceflag',
            'pooltypeid10', 'pooltypeid2', 'pooltypeid7', 'poolcnt']

num_vars = [i for i in vars if i not in cat_vars]

num_vars.remove('parcelid')

print("number of categorical variables:", len(cat_vars))
print("number of numerical variables:", len(num_vars))

#### (1) numerical variables:

##### 집 면적에 관련된 실수 변수
- imputation을 어떻게 할 거라는 설명!

In [ ]:
def num_imputation(df):
    for var in num_vars:
        if var in ["garagecarcnt", "numberofstories"]:
            df[var] = df[var].fillna(1)
        elif var in ["buildingqualitytypeid", "roomcnt", "bedroomcnt", "unitcnt", 'bathroomcnt', 
                     'calculatedbathnbr', 'fullbathcnt', 'threequarterbathnbr', 'yearbuilt']:
            a = df[var].mode()
            df[var] = df[var].fillna(int(a[0]))
        elif var in ['finishedfloor1squarefeet', 'calculatedfinishedsquarefeet',
                     'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15',
                     'finishedsquarefeet50', 'finishedsquarefeet6', 'garagetotalsqft', 'lotsizesquarefeet',
                     'taxvaluedollarcnt', 'landtaxvaluedollarcnt', 'structuretaxvaluedollarcnt', 'taxamount',
                     'latitude', 'longitude']:
            df[var] = df[var].fillna(df[var].mean())
        else:    # ['poolsizesum', 'fireplacecnt', 'yardbuildingsqft17', 'yardbuildingsqft26', "poolsizesum","taxdelinquencyyear_past"]
            df[var] = df[var].fillna(0) 

In [ ]:
num_imputation(merged_1)

#### (2) dummy/categorical variables:

In [ ]:
def cat_imputation(df):
    for var in cat_vars:                  
        if var in ['taxdelinquencyyear', 'assessmentyear']:
            df[var] = df[var].fillna(0)
        elif var == "poolcnt":
            df[var] = df[var].replace(1, True)
            df[var] = df[var].fillna(False)            
        elif var == "decktypeid":
            df[var] = df[var].replace('66.0', True)
            df[var] = df[var].fillna(False)         
        else:
            a = df[var].mode()
            df[var] = df[var].fillna(a[0])

In [ ]:
cat_imputation(merged_1)

#### (3) imputation 후 missing value ratio 다시 체크

In [ ]:
merged_1_des = merged_1.describe(include='all').transpose()
merged_1_des["missing_ratio"] = 1-(merged_1_des["count"]/merged_1.shape[0])
merged_1_des["dtype"] = merged_1.dtypes
merged_1_missing = pd.DataFrame(merged_1_des["missing_ratio"].sort_values(ascending=False))
merged_1_missing.reset_index()

In [ ]:
merged_2017 = merged_1.copy()

In [ ]:
merged_2016["taxdelinquencyyear"]

### 3.2 Feature Correlation

#### 3.2.1 Numerical Features

#### 3.2.2 Categorical Features

### 3.3 Determination of outliers and variables using OLS model

#### 3.3.1 Model by all numerical Features

#### 3.3.2 Model by all categorical Features

#### 3.3.3 Model by numerical and categorical features together

#### 3.3.4 Model by high correlation coefficient numerical features with Logerror

#### 3.3.5 Model by high correlation coefficient numerical features and categorical features 

#### 3.3.6 Multi-collinearity and variance analysis between variables

##### (1) Multi-collinearity

##### (2) Analysis of variance on categorical features

#### 3.3.7 Comparison of model performance by using variables

### 3.4 Outlier

#### 1) Option1 : IQR (Interquartile Range)

#### 2) Option2 : Standardized resids

#### 3) Option3 : Cook's Distance

#### 4) Check all outliers (option 1,2,3)

#### 5) Combination of outliers groups

### 3.5 Data preprocessing

#### 3.5.1 Select numerical variables

#### 3.5.2 Select categorical variables

## **4. Modeling**

### 4.1. Input data

### 4.2 OLS(Ordinary Least Square) Model

#### 4.2.1 Make OLS Model

#### 4.2.2 ANOVA F-test

#### 4.2.3 RMSE

#### 4.2.4 Normalization of residuals

### 4.3 Predict Logerror